In [1]:
import numpy as np
import numba

In [2]:
# import os
# os.environ["NUMBA_NUM_THREADS"] = "4"

In [3]:
import math

@numba.njit()
def compute_bin_1d_uniform(x, bins, overflow=False):
    n = bins.shape[0] - 1
    b_min = bins[0]
    b_max = bins[-1]
    if overflow:
        if x > b_max: return n-1
        elif x < b_min: return 0
    ibin = int(n * (x - b_min) / (b_max - b_min))
    if x < b_min or x > b_max:
        return -1
    else:
        return ibin

In [4]:
@numba.njit()
def numba_histogram(a, bins, weights=None,overflow=False):
    db = np.ediff1d(bins)
    is_uniform_binning = np.all(db-db[0]<1e-6)
    hist = np.zeros((len(bins)-1,), dtype=np.float64)
    a = a.flat
    b_min = bins[0]
    b_max = bins[-1]
    n = bins.shape[0] - 1
    if weights is None:
        weights = np.ones(len(a),dtype=np.float64)
    if is_uniform_binning:
        for i in range(len(a)):
            ibin = compute_bin_1d_uniform(a[i], bins, overflow=overflow)
            if ibin >= 0:
                hist[ibin] += weights[i]
    else:
        ibins = np.searchsorted(bins, a, side='left')
        for i in range(len(a)):
            # returns 0 if underflow, Nbins+1 if overflow
            ibin = ibins[i]
            if overflow:
                if ibin == n+1: ibin = n
                elif ibin == 0: ibin = 1
            if ibin >= 1 and ibin <= n:
                hist[ibin-1] += weights[i]
        pass
    return hist, bins

# @numba.njit()
@numba.njit(parallel=True)
def numba_searchsorted(bins,values):
    out = np.zeros(len(values),dtype=np.intp)
#     for i in range(len(values)):
    for i in numba.prange(len(values)):
        lo = 0
        hi = len(bins)
        while lo < hi:
            mid = (lo+hi)//2
            if bins[mid] < values[i]: lo = mid+1
            else: hi = mid
        out[i] = lo
    return out
        

In [5]:
@numba.njit()
def numba_histogram2d(ax,ay, bins_x, bins_y, weights=None,overflow=False):
    db_x = np.ediff1d(bins_x)
    db_y = np.ediff1d(bins_y)
    is_uniform_binning_x = np.all(db_x-db_x[0]<1e-6)
    is_uniform_binning_y = np.all(db_y-db_y[0]<1e-6)
    hist = np.zeros((len(bins_x)-1,len(bins_y)-1), dtype=np.float64)
    ax = ax.flat
    ay = ay.flat
    b_min_x = bins_x[0]
    b_max_x = bins_x[-1]
    n_x = bins_x.shape[0] - 1
    b_min_y = bins_y[0]
    b_max_y = bins_y[-1]
    n_y = bins_y.shape[0] - 1
    if weights is None:
        weights = np.ones(len(ax),dtype=np.float64)
    if is_uniform_binning_x and is_uniform_binning_y:
        for i in range(len(ax)):
            ibin_x = compute_bin_1d_uniform(ax[i], bins_x, overflow=overflow)
            ibin_y = compute_bin_1d_uniform(ay[i], bins_y, overflow=overflow)
            if ibin_x >= 0 and ibin_y >= 0:
                hist[ibin_x,ibin_y] += weights[i]
    else:
        ibins_x = np.searchsorted(bins_x, ax, side='left')
        ibins_y = np.searchsorted(bins_y, ay, side='left')
#         ibins_x = numba_searchsorted(bins_x, ax)
#         ibins_y = numba_searchsorted(bins_y, ay)
        for i in range(len(ax)):
            # returns 0 if underflow, N if overflow
            ibin_x = ibins_x[i]
            ibin_y = ibins_y[i]
            if overflow:
                if ibin_x == n_x+1: ibin_x = n_x
                elif ibin_x == 0: ibin_x = 1
                if ibin_y == n_y+1: ibin_y = n_y
                elif ibin_y == 0: ibin_y = 1
            if ibin_x >= 1 and ibin_y >= 1 and ibin_x <= n_x and ibin_y <= n_y:
                hist[ibin_x-1,ibin_y-1] += weights[i]
    return hist, bins_x, bins_y

In [6]:
%%time
np.random.seed(42)
N = int(1e7)
v = np.random.normal(0,1,N)
bins=np.linspace(-5,5,5000)

CPU times: user 366 ms, sys: 21.1 ms, total: 387 ms
Wall time: 386 ms


In [7]:
%%timeit -r 2 -n 1
counts,edges = np.histogram(v,bins=bins)
counts,edges

796 ms ± 1.28 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [8]:
# compile
counts,edges = numba_histogram(v,bins=bins,overflow=False)

In [9]:
%%timeit
counts,edges = numba_histogram(v,bins=bins,overflow=False)
counts,edges

40 ms ± 717 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%time
N = int(5e6)
v1 = np.random.normal(0,1,N)
v2 = np.random.normal(0,1,N)
bins=np.linspace(-3,3,50)

CPU times: user 362 ms, sys: 24 ms, total: 386 ms
Wall time: 385 ms


In [11]:
%%timeit -r 3 -n 1
np.histogram2d(v1,v2,bins=[bins,bins]);

498 ms ± 33.9 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [12]:
%%timeit
numba_histogram2d(v1,v2,bins,bins,overflow=False);

33.7 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
x = np.histogram2d(v1,v2,bins=[bins,bins]);
y = numba_histogram2d(v1,v2,bins,bins,overflow=False);
print(np.abs(x[0]-y[0]).sum())

0.0


In [14]:
np.random.seed(42)
N = int(1e7)
v1 = np.random.normal(0,1,N)
v2 = np.random.normal(0,1,N)
bins2=np.linspace(-3,3,500)
bins2[-2] = 4. # make nonuniform
bins2[-1] = 40. # make nonuniform

In [15]:
%%timeit -r 2 -n 1
np.histogram(v,bins=bins2);

726 ms ± 5.56 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [16]:
%%timeit
numba_histogram(v,bins=bins2,overflow=False);

552 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit -r 2 -n 1
np.histogram2d(v1,v2,bins=[bins2,bins2]);

1.56 s ± 30.9 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [18]:
%%timeit -r 2 -n 1
numba_histogram2d(v1,v2,bins2,bins2,overflow=False);

1.12 s ± 1.55 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [19]:
x = np.histogram2d(v1,v2,bins=[bins2,bins2]);
y = numba_histogram2d(v1,v2,bins2,bins2,overflow=False)
np.abs(x[0]-y[0]).sum()

0.0

In [20]:
%%timeit -r 3 -n 1
np.searchsorted(bins2, v1, side='left')

754 ms ± 4.14 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [21]:
%%timeit -r 3 -n 1
numba_searchsorted(bins2,v1)

The slowest run took 4.01 times longer than the fastest. This could mean that an intermediate result is being cached.
242 ms ± 167 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [22]:
(np.searchsorted(bins2, v1, side='left')-numba_searchsorted(bins2,v1)).sum()

0